### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [9]:
num_iterations = 1
empty_list = []
for i in range(1100, 1200):
    if i == 724:
        continue

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.1979285733412973
cusum = 0.11337395828824036
eta = 2.4991300131320346
hl_amp_ratio = 0.6893504281056604
kurtosis = -1.0882757314904505
n_points = 406
period = 0.6948863781634456
period_SNR = 119.40144566591988
period_log10FAP = -60.22626001909631
period_uncertainty = 0.00023489119692277072
phase_cusum = 0.28595477721418905
phase_eta = 0.2330626760902631
phi21 = 0.9851904040910979
phi31 = 2.19752103883966
quartile31 = 0.2941628871208355
r21 = 0.36888143430147086
r31 = 0.20032653759389293
shapiro_w = 0.9485974311828613
skewness = -0.30691129294079716
slope_per10 = -0.20685358270006232
slope_per90 = 0.18290869094800163
stetson_k = 0.8105753110083629
weighted_mean = 14.453977495868315
weighted_std = 0.15650969137123683
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.46


######### Lightcurve features #########

amplitude = 0.12812461251309387
cusum = 0.09581188774900751
eta = 1.9124440384164767
hl_amp_ratio = 1.0433398990881495
kurtosis = -0.6592715548104584
n_points = 429
period = 0.4760510746381408
period_SNR = 113.56436838737486
period_log10FAP = -55.868297955171805
period_uncertainty = 9.214367678819779e-05
phase_cusum = 0.29981048573938396
phase_eta = 0.720334175641085
phi21 = 0.31042001268713126
phi31 = 1.3133259899514078
quartile31 = 0.1753001077407994
r21 = 0.0967588458827197
r31 = 0.009700906526539627
shapiro_w = 0.9824309945106506
skewness = 0.25052442213831133
slope_per10 = -0.07606073544928643
slope_per90 = 0.14887343552274715
stetson_k = 0.8199850659931508
weighted_mean = 14.544069442810255
weighted_std = 0.10165027519862752


 ################################################## error in index 1

...



######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.24933583522193153
cusum = 0.12961576865118016
eta = 2.2064306725755753
hl_amp_ratio = 0.48780937129652235
kurtosis = -0.7669376738880311
n_points = 838
period = 0.6257477364071711
period_SNR = 188.06468865798772
period_log10FAP = -109.97241891238194
period_uncertainty = 0.00013858694318524822
phase_cusum = 0.3042376514007166
phase_eta = 0.38395603941602885
phi21 = -2.4136807987457254
phi31 = -4.409413815606644
quartile31 = 0.3394189690168794
r21 = 0.43330210223127535
r31 = 0.2559982903022099
shapiro_w = 0.9581802487373352
skewness = -0.38196862420805494
slope_per10 = -0.060390325684980255
slope_per90 = 0.05967347666299811
stetson_k = 0.7503357599265604
weighted_mean = 15.749126907967428
weighted_std = 0.20578753409950967
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.62


######### Lightcurve features #########

amplitude = 0.3918226025621389
cusum = 0.9091604176488641
eta = 9.873374249368341
hl_amp_ratio = 1.0749308594913622
kurtosis = 14.439052596303505
n_points = 788
period = 3532.0330981502775
period_SNR = 58.33815132611802
period_log10FAP = -21.74232914944636
period_uncertainty = 2825.626478520222
phase_cusum = 0.9091604176488643
phase_eta = 9.885095930555652
phi21 = 1.6655848678891259
phi31 = 1.3368186159518376
quartile31 = 0.30234459840256633
r21 = 0.6718111015550278
r31 = 0.502493585546298
shapiro_w = 0.6552150249481201
skewness = 3.3988961243946094
slope_per10 = -52.69104292401318
slope_per90 = 40.77480285509144
stetson_k = 0.6958027857503597
weighted_mean = 9.945504767446275
weighted_std = 0.19435435484162264
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4488476876371281
cusum = 0.20954164020563099
eta = 1.7881941425475945
hl_amp_ratio = 0.3043046640531248
kurtosis = -0.7017845394917375
n_points = 2227
period = 0.5646537511334606
period_SNR = 253.47926787096472
period_log10FAP = -280.9750667917802
period_uncertainty = 0.00011073379588544263
phase_cusum = 0.3636122150923276
phase_eta = 0.4879913947404674
phi21 = 0.7047008814291693
phi31 = -1.321072217468538
quartile31 = 0.6209094043465342
r21 = 0.46687046528265114
r31 = 0.2348490027681685
shapiro_w = 0.8909294009208679
skewness = -0.755018969018929
slope_per10 = -0.01594827962411492
slope_per90 = 0.01738894465278256
stetson_k = 0.6883360144529356
weighted_mean = 13.758026465908648
weighted_std = 0.41219379548772855
